# Web Scraping

In [165]:
import requests
from bs4 import BeautifulSoup

urls = ["https://www.cnbctv18.com/economy/budget-24-gdp-gst-collection-fiscal-defecit-painting-a-picture-of-a-resilient-india-saurabh-m-deshmukh-18982831.htm", "https://www.aljazeera.com/economy/2024/2/5/analysis-indias-2024-interim-budget-shows-a-changing-economy"]

text_arr = []
for i in urls :
    req = requests.get(i)
    soup = BeautifulSoup(req.content, "html.parser").get_text()
    text_arr.append(soup)
text_joined = '\n'.join(text_arr)
# text_joined

In [166]:
## removing \n from text

import re
text = re.sub(r"\n", "", text_joined)
# text

In [153]:
## Splitting total text into sentences

import re

match = r"\.[^\d]"
sentences = re.split(match, text)

# Model 1:

### Imports

In [186]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from termcolor import colored, cprint
import re

### Loading the model

In [187]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Color coding entities

In [188]:
total_text = ''
for text in sentences :
    # print('\n---------------------------------------',text,'---------------------------------------------------------\n')
    ner_results = nlp(text)
    colored_text = ""
    
    ner_data_sorted = sorted(ner_results, key=lambda x: x['start'])
    current_position = 0
    
    color = ''
    for entity in ner_data_sorted:
        prev = color
        start = entity['start']
        end = entity['end']
        word = entity['word']
        entity_type = entity['entity'].split("-")[1]  # Extract entity type
        colored_text += text[current_position:start]
        if(entity_type == 'PER'):
            color = 'red'
        elif(entity_type == 'LOC'):
            color = 'cyan'
        elif(entity_type == 'ORG'):
            color = 'yellow'
        else:
            color = 'blue'
        current_position = end
        if(color != '' and prev != color):
            colored_text += colored(" "+entity_type+" ",'black','on_'+color) + " "
        colored_text += colored(text[start:end], color, attrs=['bold'])
        
    colored_text += text[current_position:]
    total_text += colored_text

### Result

In [189]:
print(total_text)

Budget '24 — How it paints a picture of a resilient  LOC  IndiaJoin UsLanguageEnglishहिन्दीLatest newsMarketsEconomyPersonal FinanceIndiaWorldSpecial CoverageWorld Cup 2023G20 CoverageDiwali 2023Chandrayaan 3Election 2023Assembly Elections 2023Election AnalysisLok Sabha Election 2024Upcoming EventsEngage with  ORG  CNBC-TV18Photos VideosMinis Web Stories PollsQuizHealthcarePoliticsTravelEducation & CareersSportsViewsAutoEntertainment TechnologyStartups LifestyleRetail  Real EstateMinisIndia Business Leader AwardsBranded ContentThe Growth SummitCeo awardsAccelerate Your Cloud JourneyAccelerating to a Connected FutureFinancial Services Cloud SymposiumCryptoEducation NextEy Entrepreneur Of The YearIRMAWizards of financeThe Thought LeagueDiscover CNBCTV18AboutContactAdvertiseDisclaimerTerms of UsePrivacy PolicyShowsAnchorsPolls11:11 NewsletterLife  Watch Live TV MarketPersonal FinanceBusinessEconomyFeaturedNextGenLIVE TVNewCNBC-TV18 EdgeNewSME Champion AwardsNewLatest NewsLiveMarket LiveNe

# Model 2:
##### https://huggingface.co/Babelscape/wikineural-multilingual-ner?text=George+Washington+went+to+Washington

### Loading the model

In [190]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

### Color coding entities

In [191]:
total_text = ''
for text in sentences :
    # print('\n---------------------------------------',text,'---------------------------------------------------------\n')
    ner_results = nlp(text)
    colored_text = ""
    
    ner_data_sorted = sorted(ner_results, key=lambda x: x['start'])
    current_position = 0
    
    color = ''
    for entity in ner_data_sorted:
        prev = color
        start = entity['start']
        end = entity['end']
        word = entity['word']
        entity_type = entity['entity_group']  # Extract entity type
        colored_text += text[current_position:start]
        if(entity_type == 'PER'):
            color = 'red'
        elif(entity_type == 'LOC'):
            color = 'cyan'
        elif(entity_type == 'ORG'):
            color = 'yellow'
        else:
            color = 'blue'
        current_position = end
        if(color != '' and prev != color):
            colored_text += colored(" "+entity_type+" ",'black','on_'+color) + " "
        colored_text += colored(text[start:end], color, attrs=['bold'])
        
    colored_text += text[current_position:]
    total_text += colored_text

### Result

In [192]:
print(total_text)

Budget '24 — How it paints a picture of a resilient  LOC  IndiaJoin UsLanguageEnglishहिन्दीLatest newsMark MISC  etsEconomyPersonal FinanceIndiaWorldSpecial CoverageWorld Cup 2023G20 CoverageDiwali 2023Chandrayaan 3Election 2023Assembly Elections 2023Election AnalysisLok Sabha Election 2024Upcoming EventsEngage with CNBC-TV18Photos VideosMinis Web Stories PollsQuizHealthcarePoliticsTravelEducation & CareersSportsViewsAutoEntertainment TechnologyStartups LifestyleRetail  Real EstateMinisIndia Business Leader AwardsBranded ContentThe Growth SummitCeo awardsAccelerate Your Cloud JourneyAccelerating to a Connected FutureFinancial Services Cloud SymposiumCryptoEducation NextEy Entrepreneur Of The YearIRMAWizards of financeThe Thought LeagueDiscover CNBCTV18AboutContactAdvertiseDisclaimerTerms of UsePrivacy PolicyShowsAnchorsPolls11:11 NewsletterLife  Watch Live TV MarketPersonal FinanceBusinessEconomyFeaturedNextGenLIVE TVNewCNBC-TV18 EdgeNewSME Champion AwardsNewLatest NewsLiveMarket LiveN

# Conclusion
#### By identifying and labeling named entities present in the website's content, we gain valuable insights into the structure and context of the text.